# ArcFace Loss (Additive Angular Margin Loss)

## Теория ArcFace

В случае с обучением на задачу классификации первая подходящая лосс-функция, которая нам приходит в голову — Cross-Entropy. И на ней действительно можно обучать сеть для распознавания лиц. Но за много лет люди придумали более хитрые лосс-функции, которые делают обучение сети для распознавания лиц более эффективным. Одним из лучших считается ArcFace Loss (Additive Angular Margin Loss).

Этот лосс — чуть измененная кросс-энтропия. Он позволяет достичь лучшего распределения векторов лиц на сфере. В нем добавлены некоторые дополнительные ограничения и гиперпараметры, для того чтобы эмбеддинги лиц одного класса были более близки между собой, а эмбеддинги лиц разных людей оставались далеки. То есть, этот лосс позволяет лучше кластеризовать лица на сфере единичного радиуса.


**Как устроен ArcFace**:

Стандартные SoftMax + кросс-энтропия (CE) выглядят так:

$$L_{CE} = \frac{-1}{N}\sum_1^N \frac{e^{W_{y_i}^{T}x_i + b_{y_i}}}{\sum^n_{j=1}e^{W_j^Tx_i+b_j}},$$

здесь:
- $x_i \in \mathbb{R^d}$ — вектор $i$-го элемента обучающей выборки перед последним полносвязным слоем сети. $y_i$ — класс этого элемента;
- $W_j \in \mathbb{R^d}$ — j-ый столбец матрицы весов последнего слоя сети (т.е. слоя, который производит итоговую классификацю входящего объекта);
- $b_j \in \mathbb{R^d}$ — j-ый элемент вектора байеса последнего слоя сети;
- $N$ — batch size;
- $n$ — количество классов.


Хотя этот лосс работает хорошо, он явным образом не заставляет эмбеддинги $x_i$ элементов, принадлежащих одному классу, быть близкими друг к другу по расстоянию. И не заставляет эмбеддинги элементов, принадлежащих разным классам, быть далеко друг от друга. Все, что хочет этот лосс — чтобы на основе эмбеддингов $x_i$ можно было хорошо классифицировать элементы, никакие ограничений на расстояния между эмбеддингами $x_i$ он не вводит.

Из-за этого у нейросетей для распознавания лиц, которые обучены на обычном CE loss, бывают проблемы с распознаванием лиц, которые сильно отличаются от лиц того же человека разными допатрибутами (шляпа/прическа/очки и т.п.). Просто эмбеддинг для таких лиц получается довольно далек по расстоянию от других эмбеддингов лиц этого же человека.

Давайте теперь немного поправим формулу:
- уберем байес последнего слоя, т.е. сделаем $b_j=0$;
- нормализуем веса последнего слоя: ||$W_j$|| = 1;
- нормализуем эмбеддинги: ||$x_i$|| = 1. Перед подачей их на вход последнему слою (т.е. перед умножением на матрицу $W_j$) умножим их на гиперпараметр s. По сути, мы приводим норму всех эмбеддингов к s. Смысл этого гиперпараметра в том, что, возможно, сети проще будет классифицировать эмбеддинги, у которых не единичная норма.

Нормализация эмбеддингов приводит к тому, что эмбеддинги начинают быть распределены по сфере единичного радиуса (и сфере радиуса s после умножения на ниперпараметр s). И итоговые предсказания сети после последнего слоя зависят только от угла между эмбеддингами $x_i$ и выученных весов $W_j$. От нормы эмбеддинга $x_i$ они больше не зависят, т.к. у всех эмбеддингов они теперь одинаковые.

Получается, в степени экспоненты у нас останется выражение $s W_{y_i}^{T}x_i$, которое можно переписать в виде  $s W_{y_i}^{T}x_i = s ||W_{y_i}||\cdot ||x_i|| \cdot cos\Theta_{y_i}$. Тут $\Theta_{y_i}$ — это угод между векторами $W_{y_i}$ и $x_i$. Но так как мы сделали нормы $W_{y_i}$ и $x_i$ единичными, то все это выражение просто будет равно $s cos\Theta_{y_i}$.

В итоге мы получим следующую формулу лосса:

$$L = \frac{-1}{N}\sum_1^N \frac{e^{s\ cos\Theta_{y_i}}}{e^{s\ cos\Theta_{y_i}} + \sum^n_{j=1,\ j\ne y_i} e^{s\ cos\Theta_j}}$$


И последний шаг. Добавим еще один гиперпараметр $m$. Он называется additive angular margin penalty и заставляет эмбеддинги одного класса быть ближе друг к другу, а эмбеддинги разных классов — более далекими друг от друга.

В итоге получим вот что:

$$L_{ArcFace} = \frac{-1}{N}\sum_1^N \frac{e^{s\ cos(\Theta_{y_i} + m)}}{e^{s\ cos(\Theta_{y_i} + m)} + \sum^n_{j=1,\ j\ne y_i} e^{s\ cos\Theta_j}}$$

Это и есть ArcFace Loss с двумя  гиперпараметрами, s и m.

Получается, что ArcFace Loss завтавляет сеть выучивать эмбеддинги, распределенные по сфере радиуса s, причем чтобы эмбеддинги одного класса были ближе друг к другу, а эмбеддинги разных классов — более далеки друг от друга.



**Доплитература по ArcFace Loss:**

Оригинальная статья: https://arxiv.org/pdf/1801.07698.pdf

## Другие лоссы

Кроме ArcFace, есть еще много разных вариантов лоссов для задачи face recognition. Некоторые из них можно найти, например, [тут](https://openaccess.thecvf.com/content_CVPRW_2020/papers/w48/Hsu_A_Comprehensive_Study_on_Loss_Functions_for_Cross-Factor_Face_Recognition_CVPRW_2020_paper.pdf). Вы можете попробовать реализовать другие лосс-функции в этом проекте в качестве дополнительного задания.

Кроме этого, можно миксовать лосс-функции. Например, обучать нейросеть на сумме ArcFace и TripletLoss. Часто так выходит лучше, чем если обучать на каком-то одном лоссе.

# Датасет

В качестве датасета нужно использовать картинки из CelebA, выровненные при помощи своей модели из задания 1. Очень желательно их еще кропнуть таким образом, чтобы нейросети поступали на вход преимущественно только лица без какого либо фона, частей тела и прочего. Целиком брать весь датасет CelebA не обязательно, он слишком большой.

Если планируете делать дополнительное задание на Identificaton rate metric, то **обязательно разбейте заранее датасет на train/val или train/val/test.** Это нужно сделать не только на уровне кода, а на уровне папок, чтобы точно знать, на каких картинках модель обучалась, а на каких нет. Лучше заранее почитайте [ноутбук с заданием](https://colab.research.google.com/drive/1sjO2-N8EsLb2HQcOELikFQCqKUWrPehJ?usp=sharing).

# План заданий

Итак, вот, что от вас требуется в этом задании:

* Выбрать модель (или несколько моделей) для обучения. Можно брать предобученные на ImageNet, но нельзя использовать модели, предобученные на задачу распознавания лиц.
* Обучить эту модель (модели) на CE loss. Добиться accuracy > 0.7.
* Реализовать ArcFace loss.
* Обучить модель (модели) на ArcFace loss. Добиться accuracy > 0.7.
* Написать небольшой отчет по обучению, сравнить CE loss и ArcFace loss.

**P.S. Не забывайте сохранять модели после обучения**

### Импорты

In [1]:
import torch.nn as nn
import torch
import torch.optim as optim
from torch.nn import MSELoss
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
from torchvision.models import resnet18, resnet34, resnet101, efficientnet_b0
import torch
import os
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR
import math
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from models import get_recognition_model
from defenitions import ArcFaceLoss

### Загрузка данных

In [2]:
aligned_root = './data/celeba_aligned_top_200'

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.9, 1.1)), 
    transforms.ToTensor(),
])

val_transform = transforms.Compose([transforms.ToTensor(), ])



dataset = datasets.ImageFolder(root=aligned_root)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataset.dataset.transform = transform
val_dataset.dataset.transform = val_transform      

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

n_classes = len(dataset.classes)

facerec_model = get_recognition_model()
facerec_model.fc = nn.Linear(in_features=512, out_features=n_classes)
facerec_model = facerec_model.to(device)


### Обучение Face Recognition CE loss

In [3]:
loss_fn = nn.CrossEntropyLoss()
epochs = 15

print(f"Total {n_classes} classes, default accuracy is {100* 1/n_classes:.4f}")

# до того как изобрел scheduler
lrs = [0.001, 0.0001]

for lr in lrs:
    optimizer = torch.optim.AdamW(facerec_model.parameters(), lr=lr)
    for epoch in range(epochs):
        facerec_model.train()
        epoch_train_loss = 0
        epoch_train_correct = 0
        epoch_train_total = 0
        for imgs, labels in tqdm(train_loader):
            imgs = imgs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = facerec_model(imgs)
            loss = loss_fn(out, labels)
            epoch_train_loss += loss.item()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(out.data, 1)
            epoch_train_total += labels.size(0)
            epoch_train_correct += (predicted == labels).sum().item()
            epoch_train_loss += loss.item() * imgs.size(0)


        facerec_model.eval()
        epoch_val_loss = 0
        epoch_val_correct = 0
        epoch_val_total = 0
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            out = facerec_model(imgs)
            loss = loss_fn(out, labels)
            epoch_val_loss += loss.item()

            _, predicted = torch.max(out.data, 1)
            epoch_val_total += labels.size(0)
            epoch_val_correct += (predicted == labels).sum().item()
            epoch_val_loss += loss.item() * imgs.size(0)

        avg_train_loss = epoch_train_loss / len(train_loader)
        avg_val_loss = epoch_val_loss / len(test_loader)
        print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        print(f'Train acc: {100 * epoch_train_correct/epoch_train_total:.4f}, Val acc: {100* epoch_val_correct/epoch_val_total:.4f}')
        torch.save(facerec_model.state_dict(), f'./models/face_rec_CE_{n_classes}_classes.pth')

        

Total 200 classes, default accuracy is 0.5000


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1: Train Loss: 314.9622, Val Loss: 328.5762
Train acc: 4.9338, Val acc: 3.3184


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 2: Train Loss: 221.1519, Val Loss: 220.7007
Train acc: 20.1615, Val acc: 19.0135


...

Epoch 13: Train Loss: 0.0419, Val Loss: 76.0418
Train acc: 99.9776, Val acc: 74.4395


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 14: Train Loss: 0.0354, Val Loss: 76.5682
Train acc: 99.9551, Val acc: 74.1704


  0%|          | 0/70 [00:01<?, ?it/s]

Epoch 15: Train Loss: 0.0339, Val Loss: 76.7036
Train acc: 99.9551, Val acc: 74.0807


### ArcLoss

In [5]:
device

device(type='cuda')

In [6]:
import itertools 

facerec_model = resnet34(weights='DEFAULT')
facerec_model.fc = nn.Sequential(
    nn.Dropout(p=0.4, inplace=True),
    nn.Linear(in_features=512, out_features=512)
)
facerec_model.to(device) 

loss_fn = ArcFaceLoss(n_classes, 512, 0.3, 64).to(device)
epochs = 20

print(f"Total {n_classes} classes, default accuracy is {100* 1/n_classes:.4f}")

optimizer = torch.optim.AdamW(
    itertools.chain(facerec_model.parameters(), loss_fn.parameters()), 
    lr=0.0005
)
scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-5) 


# facerec_model.load_state_dict(torch.load('face_recognition.pth', map_location=device))


for epoch in range(epochs):
    facerec_model.train()
    epoch_train_loss = 0
    epoch_train_correct = 0
    epoch_train_total = 0
    for imgs, labels in tqdm(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = facerec_model(imgs)
        loss = loss_fn(out, labels)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()

        embeddings_norm = F.normalize(out, p=2, dim=1)
        W_norm = F.normalize(loss_fn.W, p=2, dim=1)

        logits = torch.mm(embeddings_norm, W_norm.T) * loss_fn.scale
        _, predicted = torch.max(logits, 1)
        epoch_train_total += labels.size(0)
        epoch_train_correct += (predicted == labels).sum().item()

    scheduler.step()

    facerec_model.eval()
    epoch_val_loss = 0
    epoch_val_correct = 0
    epoch_val_total = 0
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        out = facerec_model(imgs) # embeddings
        loss = loss_fn(out, labels)
        epoch_val_loss += loss.item()

        embeddings_norm = F.normalize(out, p=2, dim=1)
        W_norm = F.normalize(loss_fn.W, p=2, dim=1)

        logits = torch.mm(embeddings_norm, W_norm.T) * loss_fn.scale
        _, predicted = torch.max(logits, 1)
        epoch_val_total += labels.size(0)
        epoch_val_correct += (predicted == labels).sum().item()

    avg_train_loss = epoch_train_loss / len(train_loader)
    avg_val_loss = epoch_val_loss / len(test_loader)
    print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
    print(f'Epoch {epoch+1}: LR = {optimizer.param_groups[0]["lr"]:.6f}')
    print(f'Train acc: {100 * epoch_train_correct/epoch_train_total:.4f}, Val acc: {100* epoch_val_correct/epoch_val_total:.4f}')
    

Total 200 classes, default accuracy is 0.5000


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1: Train Loss: 24.4962, Val Loss: 23.2739
Epoch 1: LR = 0.000497
Train acc: 5.0460, Val acc: 9.9552


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 2: Train Loss: 22.1045, Val Loss: 22.4167
Epoch 2: LR = 0.000488
Train acc: 25.8802, Val acc: 21.2556


...

Epoch 18: Train Loss: 0.0296, Val Loss: 8.2687
Epoch 18: LR = 0.000022
Train acc: 99.9776, Val acc: 80.1794


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 19: Train Loss: 0.0273, Val Loss: 8.2709
Epoch 19: LR = 0.000013
Train acc: 99.9776, Val acc: 80.7175


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 20: Train Loss: 0.0242, Val Loss: 8.2300
Epoch 20: LR = 0.000010
Train acc: 99.9776, Val acc: 80.8072


In [7]:
torch.save(facerec_model.state_dict(), f'./models/face_rec_zadanie2.pth')
